In [240]:
%load_ext autoreload 
%autoreload 2
from os.path import expanduser
home = expanduser("~")
os.chdir(os.path.join(home,'Documents','repos','agu_data','agu_data'))
tablepath = os.path.join(home,'Documents/repos/agu_data/Notebook/Tables')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# AGU 2015: Exploratory analysis

Agu_Data
-------

I recently decided to scrap the AGU website [AGU](https://meetings.agu.org/) to look for
interesting pattern in the abstract. Here is a first analysis.


## Data 

In [260]:
from Data_Utils import *
import pandas as pd

In [281]:
data = get_all_data('agu2015')

In [282]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def presentation(time):
    time_beg = time.split('-')[0].split(':')[0]
    time_end = time.split('-')[1].split(':')[0]
    if int(time_end)-int(time_beg) == 0:
        return 'Oral'
    else:
        return 'Poster'
    
def clean_session(session):
    return ' '.join([f for f in session.split(' ') if f not in ['I','II','III','IV','V','Posters','(Half Session)']])
    
def extract_info(obj):
    ''' obj is a Paper object '''
    try:
        day = obj.date.split(',')[0].strip()
        date = obj.date.split(',')[1].strip()
        nb_authors = len(obj.authors)
        place = obj.place.split('-')[0].strip()
        room = obj.place.split('-')[-1].strip()
        ref = len(obj.reference)
        section = obj.section.strip().encode('utf-8')
        session = clean_session(obj.session.strip()).encode('utf-8')
        pres = presentation(obj.time)
        title_words = tokenizer.tokenize(repr(obj.title[0]).lower())
        invited = int('invited' in title_words)
        lentitle = len(title_words)-1 # to remove the 'u' from unicode
        return [day,date,nb_authors,place,room,ref,section,session,pres,invited,lentitle]
    except:
        return ['']

columns = ['day','date','nb_authors','place','room','ref','section','session','pres','invited','lentitle']
df = pd.DataFrame(map(extract_info,data),columns = columns).dropna()
df.to_csv(os.path.join(tablepath,'agu2015_exploratory.csv'))
    

In [278]:
df0 = df[['session','section','day']]
df0.groupby('section').session.count().sort_values(ascending=False)

section
Atmospheric Sciences                       974
Hydrology                                  945
Biogeosciences                             547
Global Environmental Change                486
Tectonophysics                             432
Volcanology, Geochemistry and Petrology    336
Seismology                                 304
Planetary Sciences                         300
SPA-Magnetospheric Physics                 292
SPA-Solar and Heliospheric Physics         261
Cryosphere                                 255
Natural Hazards                            252
Paleoceanography and Paleoclimatology      233
Earth and Planetary Surface Processes      218
Earth and Space Science Informatics        157
Education                                  150
SPA-Aeronomy                               146
Ocean Sciences                             142
Geodesy                                    135
Study of Earth's Deep Interior             129
Mineral and Rock Physics                   129
Geoma

In [291]:
from IPython.display import HTML
js = "<script type='text/javascript' src='https://public.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1020px; height: 1033px;'><noscript><a href='#'><img alt='Exploratory analysis of the AGU section this year ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Se&#47;Section_Details&#47;Story1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz' width='1020' height='1033' style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='Section_Details&#47;Story1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Se&#47;Section_Details&#47;Story1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='showVizHome' value='no' /><param name='showTabs' value='y' /><param name='bootstrapWhenNotified' value='true' /></object></div>"
from IPython.display import HTML
HTML(js)